### Examples

Vector Addition

In [76]:
import numpy as np 
from numba import cuda, float32
import math

In [77]:
@cuda.jit
def f(a, b, c):
    pos = cuda.grid(1)
    size = len(c)

    if pos < size:
        c[pos] = a[pos] + b[pos]

In [78]:
N = 100000
a = cuda.to_device(np.random.random(N))
b = cuda.to_device(np.random.random(N))
c = cuda.device_array_like(a)

forall() creates 1D grid for a given data size

In [79]:
f.forall(len(a))(a, b, c)
c.copy_to_host()

c:\Users\rouna\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 98 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([0.83095035, 0.96491347, 1.04135738, ..., 1.04118482, 1.02526096,
       1.57020609])

In [80]:
nthreads = 256
nblocks = (len(a) // nthreads) + 1
f[nblocks, nthreads](a, b, c)
c.copy_to_host()

array([0.83095035, 0.96491347, 1.04135738, ..., 1.04118482, 1.02526096,
       1.57020609])

Matrix multiplication

In [81]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B."""
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [82]:
x_h = np.arange(16).reshape([4, 4])
y_h = np.ones([4, 4])
z_h = np.zeros([4, 4])

x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)


In [83]:
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [84]:
matmul[blockspergrid, threadsperblock](x_d, y_d, z_d)

c:\Users\rouna\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [85]:
z_h = z_d.copy_to_host()
print(z_h)

[[ 6.  6.  6.  6.]
 [22. 22. 22. 22.]
 [38. 38. 38. 38.]
 [54. 54. 54. 54.]]


Better Matrix Multiplication with Shared Memory

In [86]:
TPB = 32
# threads per block

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[ty, tx] = 0
        sB[ty, tx] = 0
        if y < A.shape[0] and (tx + i * TPB) < A.shape[1]:
            sA[ty, tx] = A[y, tx + i * TPB]
        if x < B.shape[1] and (ty + i * TPB) < B.shape[0]:
            sB[ty, tx] = B[ty + i * TPB, x]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish computing
        cuda.syncthreads()
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = tmp

In [87]:
x_h = np.arange(128*128).reshape([128, 128])
y_h = np.ones([128, 128])
z_h = np.zeros([128, 128])

x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

In [88]:
threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [89]:
%%timeit
fast_matmul[blockspergrid, threadsperblock](x_d, y_d, z_d)

c:\Users\rouna\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


40.8 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [90]:
z_h = z_d.copy_to_host()
print(np.array_equal(z_h, x_h @ y_h))
print(z_h.shape)

True
(128, 128)
